In [1]:
from dotenv import load_dotenv
load_dotenv('../.vscode/.env')

True

In [2]:
from openai import OpenAI, AssistantEventHandler
import os
import json
from bs4 import BeautifulSoup
import random
import time
from typing import List, Optional

In [6]:
recipe_name = '''
Here's a delicious recipe for Zesty Herb-Stuffed Mushrooms that you can try at home:
'''
ingredients = '''
    - 24 large mushrooms, stems removed
    - 1 tablespoon olive oil
    - 1/2 cup onion, finely chopped
    - 2 cloves garlic, minced
    - 1/2 cup breadcrumbs
    - 1/4 cup grated Parmesan cheese
    - 2 tablespoons fresh parsley, chopped
    - 1 tablespoon fresh basil, chopped
    - 1 teaspoon fresh thyme leaves
    - 1/4 teaspoon red pepper flakes (optional for a zesty kick)
    - Salt and black pepper to taste
    - 2 tablespoons cream cheese, softened
    - 1/4 cup chicken or vegetable broth
'''
directions= '''
1.Preheat Oven: Set your oven to 375°F (190°C).
2.Prepare Mushrooms:
    - Clean the mushrooms with a damp towel and carefully remove the stems. Chop the stems finely.
    - Place the mushroom caps on a baking sheet, hollow-side up.
3.Cook Filling:
    - Heat olive oil in a skillet over medium heat. Add chopped onion and mushroom stems. Cook until they are soft and lightly browned, about 5-7 minutes.
    - Add garlic and cook for an additional minute until fragrant.
    - Remove from heat and transfer to a mixing bowl.
4.Mix Filling:
    - To the bowl, add breadcrumbs, Parmesan cheese, parsley, basil, thyme, red pepper flakes (if using), and salt and black pepper. Mix well to combine.
    - Stir in cream cheese until the mixture is cohesive but not too wet. If it feels dry, add a little broth to moisten.
5.Stuff Mushrooms:
    - Spoon the filling into each mushroom cap, pressing it in slightly to pack the cap.
6.Bake:
    - Drizzle the tops with a little more olive oil, if desired.
    - Bake in the preheated oven for 20 minutes, or until the mushrooms are tender and the tops are golden brown.
7.Serve:
    - Allow to cool for a few minutes before serving. These are great as an appetizer or a side dish!
'''
nutrients='''Total Carbohydrate: 50g, Dietary Fiber: 15g, Sodium: 300mg, Saturated Fat: 2g, Total Fat: 17g, Protein: 12g, Added Sugars: 0g, Total Sugars: 5g'''
total_calorie_estimation='''
1.Total Calories for entire recipe : Approximately 610 calories for the entire recipe. 
2.Calories by Ingredient
    - Large Mushrooms: 100 calories for 24 mushrooms
    - Olive Oil: 239 calories for 2 tablespoons
    - Garlic: Negligible calories
    - Fresh Parsley, Thyme, Basil: Negligible calories
    - Breadcrumbs: 200 calories for 1/2 cup
    - Parmesan Cheese: 50 calories for 1/4 cup
    - Black Pepper: Negligible calories
3.Serving Details
    - Number of People Served: 5
    - Servings per Person: 4 mushrooms each
    - Calories per Serving: The total calorie content of 610 divided by 5 people gives approximately 122 calories per serving.
'''

In [25]:
ASSISTANT_INSTRUCTION = f'''
You are helpful health-friendly recipe assistant who give different recipe each time when user ask. Recipe can be for vegetarian, vegan, and non-vegeterian diets.
For user prompt: Give me recipe
{recipe_name}
Ingredients:
{ingredients}
Instructions: 
{directions}
Nutritions:
{nutrients}
Total Calories Estimation:
{total_calorie_estimation}
Always use above format, give the total calorie estimation. Clean the response to dispaly in chat, use markdown to give the response
'''

In [26]:
print(ASSISTANT_INSTRUCTION)


You are helpful health-friendly recipe assistant who give different recipe each time when user ask. Recipe can be for vegetarian, vegan, and non-vegeterian diets.
For user prompt: Give me recipe

Here's a delicious recipe for Zesty Herb-Stuffed Mushrooms that you can try at home:

Ingredients:

    - 24 large mushrooms, stems removed
    - 1 tablespoon olive oil
    - 1/2 cup onion, finely chopped
    - 2 cloves garlic, minced
    - 1/2 cup breadcrumbs
    - 1/4 cup grated Parmesan cheese
    - 2 tablespoons fresh parsley, chopped
    - 1 tablespoon fresh basil, chopped
    - 1 teaspoon fresh thyme leaves
    - 1/4 teaspoon red pepper flakes (optional for a zesty kick)
    - Salt and black pepper to taste
    - 2 tablespoons cream cheese, softened
    - 1/4 cup chicken or vegetable broth

Instructions: 

1.Preheat Oven: Set your oven to 375°F (190°C).
2.Prepare Mushrooms:
    - Clean the mushrooms with a damp towel and carefully remove the stems. Chop the stems finely.
    - Place the

In [27]:
def show_json(obj):
    display(json.loads(obj.model_dump_json()))

In [28]:
ASSISTANT_NAME = 'Recipe Assistant'

In [29]:
client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY", ""))

In [30]:
def create_assistant(assistant_name: str, assistant_instruction:str):
    assistant = client.beta.assistants.create(
        name=assistant_name,
        instructions=assistant_instruction,
        model="gpt-4-turbo-2024-04-09",
    )
    return assistant

In [31]:
assistant = create_assistant(ASSISTANT_NAME, ASSISTANT_INSTRUCTION)

In [32]:
show_json(assistant)

{'id': 'asst_SJdW9fzLln99IBgPcE698ex5',
 'created_at': 1713758362,
 'description': None,
 'instructions': "\nYou are helpful health-friendly recipe assistant who give different recipe each time when user ask. Recipe can be for vegetarian, vegan, and non-vegeterian diets.\nFor user prompt: Give me recipe\n\nHere's a delicious recipe for Zesty Herb-Stuffed Mushrooms that you can try at home:\n\nIngredients:\n\n    - 24 large mushrooms, stems removed\n    - 1 tablespoon olive oil\n    - 1/2 cup onion, finely chopped\n    - 2 cloves garlic, minced\n    - 1/2 cup breadcrumbs\n    - 1/4 cup grated Parmesan cheese\n    - 2 tablespoons fresh parsley, chopped\n    - 1 tablespoon fresh basil, chopped\n    - 1 teaspoon fresh thyme leaves\n    - 1/4 teaspoon red pepper flakes (optional for a zesty kick)\n    - Salt and black pepper to taste\n    - 2 tablespoons cream cheese, softened\n    - 1/4 cup chicken or vegetable broth\n\nInstructions: \n\n1.Preheat Oven: Set your oven to 375°F (190°C).\n2.P

In [55]:
def create_thread():
    thread = client.beta.threads.create()
    return thread

In [80]:
create_thread()

Thread(id='thread_nYeoyTisW1qb3Vv8S01yBmJw', created_at=1713761617, metadata={}, object='thread', tool_resources=ToolResources(code_interpreter=None, file_search=None))

In [19]:
def submit_message(assistant_id, thread, user_message):
    client.beta.threads.messages.create(
        thread_id=thread.id, role="user", content=user_message
    )
    return client.beta.threads.runs.create(
        thread_id=thread.id,
        assistant_id=assistant_id,
    )


def get_response(thread):
    messages = client.beta.threads.messages.list(thread_id=thread.id)
    for m in messages:
        if m.role == 'assistant':
            return m.content[0].text.value
# Write recipe in xml format using Beautiful soup
def write_recipe(location:str,id: str, recipe_content:str):
    soup = BeautifulSoup(recipe_content, "xml")
    with open(f'{location}/{id}.xml', 'w') as f:
        f.write(soup.prettify())

def create_thread_and_run(assistant_id: str, user_input:str):
    thread = client.beta.threads.create()
    run = submit_message(assistant_id, thread, user_input)
    return thread, run

def wait_on_run(run, thread):
    while run.status == "queued" or run.status == "in_progress":
        run = client.beta.threads.runs.retrieve(
            thread_id=thread.id,
            run_id=run.id,
        )
        time.sleep(0.5)
    return run

In [33]:
thread, run = create_thread_and_run(assistant_id= 'asst_SJdW9fzLln99IBgPcE698ex5', user_input='Give me a healthy recipe for my dinner with I want to serve my family on Labor Day')

In [34]:
run = wait_on_run(thread=thread, run = run)

In [35]:
recipe_content = get_response(thread=thread)

In [57]:
thread.id

'thread_92DaZjclC1Viix9ZetosK6Hg'

In [36]:
print(recipe_content)

Here's a vibrant and nutritious recipe for **Quinoa & Black Bean Salad** perfect for a family dinner on Labor Day:

### Ingredients:
- 1 cup quinoa
- 2 cups water
- 1 can black beans, drained and rinsed
- 1 red bell pepper, diced
- 1 green bell pepper, diced
- 1 cup corn (fresh or frozen)
- 1/2 red onion, finely chopped
- 1/4 cup cilantro, chopped
- Juice of 2 limes
- 2 tablespoons olive oil
- 1 teaspoon cumin
- Salt and pepper to taste
- 1 avocado, diced (for garnish)

### Instructions:
1. **Cook Quinoa:** Rinse the quinoa under cold running water. In a saucepan, bring 2 cups of water to a boil. Add quinoa, reduce heat to low, cover, and simmer for 15 minutes or until all water is absorbed. Remove from heat and let sit, covered, for 5 minutes. Fluff with a fork and allow to cool.
2. **Prepare Salad Ingredients:** While the quinoa is cooking, prepare the black beans, bell peppers, corn, and red onion.
3. **Make Dressing:** In a small bowl, whisk together lime juice, olive oil, cumin, s

In [61]:
import openai

In [37]:
from typing_extensions import override
from openai import AssistantEventHandler
 
# First, we create a EventHandler class to define
# how we want to handle the events in the response stream.
 
class EventHandler(AssistantEventHandler):    
  @override
  def on_text_created(self, text) -> None:
    print(f"\nassistant > ", end="", flush=True)
      
  @override
  def on_text_delta(self, delta, snapshot):
    print(delta.value, end="", flush=True)
      
  def on_tool_call_created(self, tool_call):
    print(f"\nassistant > {tool_call.type}\n", flush=True)
  
  def on_tool_call_delta(self, delta, snapshot):
    if delta.type == 'code_interpreter':
      if delta.code_interpreter.input:
        print(delta.code_interpreter.input, end="", flush=True)
      if delta.code_interpreter.outputs:
        print(f"\n\noutput >", flush=True)
        for output in delta.code_interpreter.outputs:
          if output.type == "logs":
            print(f"\n{output.logs}", flush=True)
 
# Then, we use the `stream` SDK helper 
# with the `EventHandler` class to create the Run 
# and stream the response.
 
with client.beta.threads.runs.stream(
  thread_id='thread_bgz1pr4oXfSRDXTkKw5vOKyf',
  assistant_id='asst_4BHISvzWhhg7mJv8wmGs3r8c',
  instructions="Give me healthy recipe with total calories estimation",
  event_handler=EventHandler(),
) as stream:
  stream.until_done()


assistant > Here is a healthy and delicious recipe for Grilled Chicken Salad:

Ingredients:
- 2 boneless, skinless chicken breasts (each about 6 oz)
- 1 tablespoon olive oil
- Salt and pepper to taste
- 4 cups mixed salad greens (e.g., spinach, arugula, romaine)
- 1 cup cherry tomatoes, halved
- 1/2 cucumber, sliced
- 1/4 red onion, thinly sliced
- 1/4 cup crumbled feta cheese
- 2 tablespoons balsamic vinegar

Instructions:
1. Preheat the grill to medium-high heat.
2. Rub the chicken breasts with olive oil and season with salt and pepper.
3. Grill the chicken for 6-7 minutes per side or until the internal temperature reaches 165 degrees F and the juices run clear.
4. Remove the chicken from the grill and let it rest for a few minutes before slicing.
5. In a large salad bowl, combine the mixed greens, cherry tomatoes, cucumber, and red onion.
6. Add the sliced grilled chicken to the salad.
7. Sprinkle crumbled feta cheese over the top.
8. Drizzle balsamic vinegar over the salad and gen

In [50]:
from openai.types.beta.assistant_stream_event import ThreadMessageDelta

In [84]:
def stream_assistant_run(client: OpenAI, thread_id: str,assistant_id: str, prompt: str):
    stream = client.beta.threads.runs.create(
        thread_id=thread_id,
        assistant_id=assistant_id,
        instructions=prompt,
        stream=True,
        )
    for event in stream:
        if isinstance(event, ThreadMessageDelta):
            current_response = event.data.delta.content[0].text.value
            print(current_response)

In [85]:
stream_assistant_run(client, 'thread_GFibaxlQOijNkJqTnlz4RP0V', 'asst_4BHISvzWhhg7mJv8wmGs3r8c','Give me healthy recipe with total calories estimation')

ThreadMessageDelta(data=MessageDeltaEvent(id='msg_syAucKnXOQeyy1Tuap2dy0qN', delta=MessageDelta(content=[TextDeltaBlock(index=0, type='text', text=TextDelta(annotations=[], value='Sure'))], role=None), object='thread.message.delta'), event='thread.message.delta')
ThreadMessageDelta(data=MessageDeltaEvent(id='msg_syAucKnXOQeyy1Tuap2dy0qN', delta=MessageDelta(content=[TextDeltaBlock(index=0, type='text', text=TextDelta(annotations=None, value='!'))], role=None), object='thread.message.delta'), event='thread.message.delta')
ThreadMessageDelta(data=MessageDeltaEvent(id='msg_syAucKnXOQeyy1Tuap2dy0qN', delta=MessageDelta(content=[TextDeltaBlock(index=0, type='text', text=TextDelta(annotations=None, value=' Here'))], role=None), object='thread.message.delta'), event='thread.message.delta')
ThreadMessageDelta(data=MessageDeltaEvent(id='msg_syAucKnXOQeyy1Tuap2dy0qN', delta=MessageDelta(content=[TextDeltaBlock(index=0, type='text', text=TextDelta(annotations=None, value="'s"))], role=None), obje

In [1]:
import requests
import json
url = "http://localhost:8000/chat/new"

with requests.post(url, data=json.dumps({'message':'Give me a recipe'}),stream=True) as r:
    for chunk in r.iter_content(None, decode_unicode=True):
        if chunk:
            print(chunk, end='', flush=True)

data: {"role": "assistant", "content": "Sure"}

data: {"role": "assistant", "content": "!"}

data: {"role": "assistant", "content": " Let"}

data: {"role": "assistant", "content": "'s"}

data: {"role": "assistant", "content": " make"}

data: {"role": "assistant", "content": " a"}

data: {"role": "assistant", "content": " classic"}

data: {"role": "assistant", "content": " chocolate"}

data: {"role": "assistant", "content": " chip"}

data: {"role": "assistant", "content": " cookie"}

data: {"role": "assistant", "content": " recipe"}

data: {"role": "assistant", "content": ".

"}

data: {"role": "assistant", "content": "Ingredients"}

data: {"role": "assistant", "content": ":
"}

data: {"role": "assistant", "content": "-"}

data: {"role": "assistant", "content": " "}

data: {"role": "assistant", "content": "1"}

data: {"role": "assistant", "content": " cup"}

data: {"role": "assistant", "content": " uns"}

data: {"role": "assistant", "content": "alted"}

data: {"role": "assistant", "cont